In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("outputs/clusteredconstituencies.csv")
dfmatch = pd.read_excel("data/Boundary_changes_data_file.xlsx", sheet_name="All overlaps", skiprows=1)

In [3]:
df19 = pd.read_csv("data/electdata_2019.txt", header=0, delimiter=";", encoding="iso-8859-1")
df17 = pd.read_csv("data/electdata_2017.txt", header=0, delimiter=";", encoding="iso-8859-1")
df15 = pd.read_csv("data/electdata_2015.txt", header=0, delimiter=";", encoding="iso-8859-1")
df10 = pd.read_csv("data/electdata_2010ug.txt", header=0, delimiter=";", encoding="iso-8859-1")
df05 = pd.read_csv("data/electdata_2005nb.txt", header=0, delimiter=";", encoding="iso-8859-1")

dfref = pd.read_excel("data/eureferendum_constitunecy.xlsx", sheet_name="DATA", skiprows=5)
dfref = dfref.drop([0,1])

In [4]:
def get_GE_results(constituency, dfmatch=dfmatch, df19=df19, ref_party="Brexit"):
    df = dfmatch[dfmatch["New constituency name"]==constituency].copy()
    con = []
    lab = []
    lib = []
    ref = []
    grn = []
    nat = []
    oth = []
    pcs = []
    for n in range(len(df)):
        rdf = df.iloc[n]
        # print(rdf)
        # print(rdf["Percentage of old constituency in this segment (population) [notes 1 and 2]"])
        pc = rdf["Percentage of old constituency in this segment (population) [notes 1 and 2]"]
        pcs.append(pc)
        oldname = rdf["Current constituency name"]
        cdf = df19[df19["Name"]==oldname]
        # print("oldname", oldname)
        # print(cdf["CON"])

        if len(cdf)==0:
            oldname = rdf["Current constituency name"].replace("-"," ").replace(",", "")
            cdf = df19[df19["Name"]==oldname]
        if len(cdf)==0:
            onl = oldname.split()
            onl.insert(0,onl.pop(-1))
            oldname = " ".join(onl)
            cdf = df19[df19["Name"]==oldname]        
        if len(cdf)==0:
            oldname = rdf["Current constituency name"].replace("-"," ").replace(",", " ")
            onl = oldname.split()
            onl.insert(0,onl.pop(1))
            oldname = " ".join(onl)
            cdf = df19[df19["Name"]==oldname]
        if len(cdf)==0:
            oldname = rdf["Current constituency name"].replace("-"," ").replace(",", " ")
            onl = oldname.split()
            onl.insert(3,onl.pop(-1))
            oldname = " ".join(onl)
            cdf = df19[df19["Name"]==oldname]
        if len(cdf)==0:
            oldname = rdf["Current constituency name"].replace("-"," ").replace(",", " ")
            onl = oldname.split()
            onl.insert(0,onl.pop(1))
            onl.insert(3,onl.pop(-1))
            oldname = " ".join(onl)
            cdf = df19[df19["Name"]==oldname]
        if len(cdf)==0:
            oldname=rdf["Current constituency name"]
            # print(oldname)
            name_dict = {
                "City of Chester":"Chester, City of",
                "City of Durham":"Durham, City of",
                "Faversham and Mid Kent":"Faversham and Kent Mid",
                "Kingston upon Hull East":"Hull East",
                "Kingston upon Hull North":"Hull North",
                "Kingston upon Hull West and Hessle":"Hull West and Hessle",
                "The Cotswolds":"Cotswolds, The",
                "Richmond (Yorks)":"Richmond",
                "Torridge and West Devon":"Devon West and Torridge",
                "The Wrekin":"Wrekin, The",               
                "Ynys Môn":"Ynys Mon",     
                "Na h-Eileanan an Iar":"Na h-Eileanan An Iar (Western Isles)",     
                "Merthyr Tydfil and Rhymney":"Merthyr Tydfil And Rhymney"
            }
            oldname = name_dict[oldname]
            cdf = df19[df19["Name"]==oldname]
        # print("oldname", oldname)
        # print("HNE", "Carmarthen West and Pembrokeshire South")
        con.append(list(cdf["CON"])[0]*pc)
        lab.append(list(cdf["LAB"])[0]*pc)
        lib.append(list(cdf["LIB"])[0]*pc)
        if ref_party:
            ref.append(list(cdf[ref_party])[0]*pc)
        else:
            ref.append(0)
        if "Green" in cdf.columns:
            grn.append(list(cdf["Green"])[0]*pc)
        nat.append(list(cdf["NAT"])[0]*pc)
        oth.append((list(cdf["MIN"])[0] + list(cdf["OTH"])[0])*pc)
    return {
        "con":int(sum(con)),
        "lab":int(sum(lab)),
        "lib":int(sum(lib)),
        "ref":int(sum(ref)),
        "grn":int(sum(grn)),
        "nat":int(sum(nat)),
        "oth":int(sum(oth)),
        "max_pc":max(pcs)
    }
    
def get_ref_results(constituency, dfmatch=dfmatch, df19=dfref):
    df = dfmatch[dfmatch["New constituency name"]==constituency].copy()
    lve = []

    for n in range(len(df)):
        rdf = df.iloc[n]
        # print(rdf)
        # print(rdf["Percentage of new constituency in this segment (population) [notes 1 and 2]"])
        pc = rdf["Percentage of new constituency in this segment (population) [notes 1 and 2]"]
        oldname = rdf["Current constituency name"]
        cdf = df19[df19["Constituency"]==oldname]
        # print("oldname", oldname)

        if len(cdf)==0:
            oldname = rdf["Current constituency name"].replace("-"," ").replace(",", "")
            cdf = df19[df19["Constituency"]==oldname]
        if len(cdf)==0:
            onl = oldname.split()
            onl.insert(0,onl.pop(-1))
            oldname = " ".join(onl)
            cdf = df19[df19["Constituency"]==oldname]        
        if len(cdf)==0:
            oldname = rdf["Current constituency name"].replace("-"," ").replace(",", " ")
            onl = oldname.split()
            onl.insert(0,onl.pop(1))
            oldname = " ".join(onl)
            cdf = df19[df19["Constituency"]==oldname]
        if len(cdf)==0:
            oldname = rdf["Current constituency name"].replace("-"," ").replace(",", " ")
            onl = oldname.split()
            onl.insert(3,onl.pop(-1))
            oldname = " ".join(onl)
            cdf = df19[df19["Constituency"]==oldname]
        if len(cdf)==0:
            oldname = rdf["Current constituency name"].replace("-"," ").replace(",", " ")
            onl = oldname.split()
            onl.insert(0,onl.pop(1))
            onl.insert(3,onl.pop(-1))
            oldname = " ".join(onl)
            cdf = df19[df19["Constituency"]==oldname]
        if len(cdf)==0:
            oldname=rdf["Current constituency name"]
            print(oldname)
            name_dict = {
                "City of Chester":"Chester, City of",
                "City of Durham":"Durham, City of",
                "Faversham and Mid Kent":"Faversham and Kent Mid",
                "Kingston upon Hull East":"Hull East",
                "Kingston upon Hull North":"Hull North",
                "Kingston upon Hull West and Hessle":"Hull West and Hessle",
                "The Cotswolds":"Cotswolds, The",
                "Richmond (Yorks)":"Richmond",
                "Torridge and West Devon":"Devon West and Torridge",
                "The Wrekin":"Wrekin, The",               
                "Ynys Môn":"Ynys Mon",  
                "Ynys Môn":"Ynys Mon",     
                "Na h-Eileanan an Iar":"Na h-Eileanan An Iar (Western Isles)",     
                "Merthyr Tydfil and Rhymney":"Merthyr Tydfil And Rhymney"
            }
            oldname = name_dict[oldname]
            cdf = df19[df19["Constituency"]==oldname]
        print("oldname", oldname)
        print("HNE", "Carmarthen West and Pembrokeshire South")
        lve.append(list(cdf["TO USE"])[0]*pc)
    
    return sum(lve)


In [5]:
df["2019_Results"]=df["New constituency name"].apply(lambda x:get_GE_results(x))
df["2017_Results"]=df["New constituency name"].apply(lambda x:get_GE_results(x,df19=df17, ref_party="UKIP"))
df["2015_Results"]=df["New constituency name"].apply(lambda x:get_GE_results(x,df19=df15, ref_party="UKIP"))
df["2010_Results"]=df["New constituency name"].apply(lambda x:get_GE_results(x,df19=df10, ref_party="UKIP"))
df["2005_Results"]=df["New constituency name"].apply(lambda x:get_GE_results(x,df19=df05, ref_party=None))
df["Ref16_Results"] = df["New constituency name"].apply(lambda x:get_ref_results(x))
df["Low_Confidence_Imputation"] = df["2019_Results"].apply(lambda x: 0 if x["max_pc"] >0.8 else 1)

oldname Aberavon
HNE Carmarthen West and Pembrokeshire South
oldname Bridgend
HNE Carmarthen West and Pembrokeshire South
oldname Neath
HNE Carmarthen West and Pembrokeshire South
oldname Ogmore
HNE Carmarthen West and Pembrokeshire South
oldname Aldershot
HNE Carmarthen West and Pembrokeshire South
oldname North East Hampshire
HNE Carmarthen West and Pembrokeshire South
oldname Aldridge-Brownhills
HNE Carmarthen West and Pembrokeshire South
oldname Walsall South
HNE Carmarthen West and Pembrokeshire South
oldname Altrincham and Sale West
HNE Carmarthen West and Pembrokeshire South
oldname Alyn and Deeside
HNE Carmarthen West and Pembrokeshire South
oldname Delyn
HNE Carmarthen West and Pembrokeshire South
oldname Amber Valley
HNE Carmarthen West and Pembrokeshire South
oldname Mid Derbyshire
HNE Carmarthen West and Pembrokeshire South
oldname Arundel and South Downs
HNE Carmarthen West and Pembrokeshire South
oldname Bognor Regis and Littlehampton
HNE Carmarthen West and Pembrokeshire

In [6]:

df["Low_Confidence_Imputation"].value_counts()

Low_Confidence_Imputation
0    437
1    195
Name: count, dtype: int64

In [7]:
df["2019_Results"][628]

{'con': 8124,
 'lab': 14873,
 'lib': 2429,
 'ref': 0,
 'grn': 0,
 'nat': 20206,
 'oth': 455,
 'max_pc': 0.7242329096788749}

In [8]:
df.to_csv("outputs/constituencieswithresults.csv",index=False)
